In [300]:
# importing all packages that may be required
import os
#import gc
import re
import csv
import html
import copy
#import tqdm
import time
import mailbox
import unicodedata
import pandas as pd
#import multiprocessing
from bs4 import BeautifulSoup
from dask import dataframe as dd
from dask.multiprocessing import get
from nltk.stem import WordNetLemmatizer as wnl
from nltk import pos_tag as pt
from nltk.corpus import wordnet as wn

In [249]:
cwd = os.getcwd()

In [243]:
# importing mailbox
mail_container = mailbox.mbox(cwd+'/raw_data/podesta-emails.mbox-2016-11-06')

In [244]:
# making working copy
mail_cp = copy.copy(mail_container)

In [62]:
# creating function to extract body and other tags & write to a csv file (9.18Gb)
def extract(mail_cp):
    try:
        with open('podesta_mailbox.csv', 'w', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow(['date', 'from', 'to', 'subject', 'originator_org', 'spam_header', 'msg_body'])
            for mail in mail_cp.itervalues():
                msg=mail
                if mail.is_multipart():
                    msg_content = ''.join(str(part.get_payload(decode=True)) for part in mail.walk())
                else:
                    msg_content = mail.get_payload(decode=True)
                writer.writerow([mail['Date'], mail['From'], mail['To'],
                                 mail['Subject'], mail['X-OriginatorOrg'],
                                 mail['SpamDiagnosticMetadata'],msg_content])
    except:
        pass

In [248]:
# running function and display time
%time extract(mail_cp)
# 15 mins - 9.18 gb

1092.792960882187


In [ ]:
# deleting unwanted variables
%who
del extract
del mail_cp
del mailbox
del mail_container

In [ ]:
# spam headers = https://docs.microsoft.com/en-us/office365/securitycompliance/anti-spam-message-headers

In [133]:
# listing datatypes of dataframe and including in importing csv with 50Mb blocksize
dtypes={'date': 'object', 'from': 'object', 'to': 'object', 'subject': 'object', 'originator_org': 'object', 'spam_header': 'object','msg_body': 'object'}
podesta = dd.read_csv(cwd+'/podesta_mailbox.csv', dtype=dtypes, blocksize=50e6)

In [134]:
podesta.head(2)

,date,from,to,subject,originator_org,spam_header,msg_body
0,"Fri, 3 Apr 2015 14:06:30 +0000",Tina Flournoy <Tina@presidentclinton.com>,John Podesta <john.podesta@gmail.com>,just talked to him,NaN,NaN,b'\n\nSC was the first thing he mentioned.\nTo...
1,"Mon, 22 Feb 2016 23:00:30 +0000",Ira Shapiro <ira@shapiroglobal.com>,John Podesta <john.podesta@gmail.com>,today's fundraiser,shapiroglobal.com,NSPM,"Noneb""John,\n\nSorry we didn't get to visit th..."


In [50]:
podesta.npartitions

184

In [190]:
%who
del Client
del client
del dtypes

BeautifulSoup	 Client	 clean	 client	 copy	 csv	 cwd	 dask	 dd	 
dtypes	 gc	 html	 mailbox	 multiprocessing	 os	 pd	 re	 saxutilis	 
time	 tqdm	 unescape	 unicodedata	 xml	 


In [52]:
# counting number of rows in dataframe
%time podesta.index.count().compute(scheduler='processes')


CPU times: user 1.34 s, sys: 108 ms, total: 1.45 s
Wall time: 56.1 s


50887

In [135]:
# getting a sample email body to create text cleaning function
podesta.head().msg_body[0:10][2]

'Noneb\'\\n\\nSent from my iPad\\n\\nBegin forwarded message:\\n\\nFrom: "Knaebel, Sergio" <SKnaebel@sandlerfoundation.org<mailto:SKnaebel@sandlerfoundation.org>>\\nDate: June 16, 2014 at 6:21:46 PM PDT\\nTo: "Sandler, Herbert" <hms@sandlerfoundation.org<mailto:hms@sandlerfoundation.org>>, "Daetz, Steve" <sdaetz@sandlerfoundation.org<mailto:sdaetz@sandlerfoundation.org>>\\nSubject: NYTimes: The True Cost of Hidden Money\\n\\n\\nhttp://nyti.ms/1kEeCfj\\n\\nNew research suggests that tax evasion deepens inequality.\\n\\n(by Picketty\\\'s protege )\\n\\nSergio Knaebel\\n\'b\'<html>\\n<head>\\n<meta http-equiv="Content-Type" content="text/html; charset=us-ascii">\\n</head>\\n<body dir="auto">\\n<div><br>\\n<br>\\nSent from my iPad</div>\\n<div><br>\\nBegin forwarded message:<br>\\n<br>\\n</div>\\n<blockquote type="cite">\\n<div><b>From:</b> &quot;Knaebel, Sergio&quot; &lt;<a href="mailto:SKnaebel@sandlerfoundation.org">SKnaebel@sandlerfoundation.org</a>&gt;<br>\\n<b>Date:</b> June 16, 2014

In [305]:
# deleting everything between "<>" html tags in the email body
# removing html entities and normalizing text also tagging and lemmatizing text

def get_wordnet_pos(word):
    if word.startswith('J'):
        return wn.ADJ
    elif word.startswith('V'):
        return wn.VERB
    elif word.startswith('N'):
        return wn.NOUN
    elif word.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN


def clean(msg_content):
    l=[]
    msg_clean = re.sub(r'<(.*?)>', '', msg_content).replace('\\n', ' ').replace('None', '')
    msg_clean = re.sub("[/\\\:?\"]", '', re.sub(r' +', ' ', unicodedata.normalize('NFKD', html.unescape(msg_clean))))
    msg_clean = nltk.word_tokenize(msg_clean)
    ptag = pt(msg_clean)
    for word in ptag:
        lem = wnl().lemmatize(word[0], get_wordnet_pos(word[1]))
        if lem!=None or lem!='' or lem!=' ':
            l.append(str(lem.lower()))
        else:
            pass
    return ' '.join(w for w in l)

In [306]:
clean(podesta.head().msg_body[1])

"bjohn , sorry we do n't get to visit this morning . i teach once a semester for madeleine , and this be the morning , so i have to exit early . it be great that you be there for katie , but i 'm not surprise . having be the beneficiary of your support when i run for congress , i know that you 're always there for former colleague and old friend . the republicans seem to be move along their remarkable path , set up a clash between one of the least qualified presidential candidate in history with one of the most qualified . given the state of the country , it promise to be quite a battle . one that we have to win . all my best , ira ira shapiro ira shapiro global strategies , llc 1200 new hampshire avenue , nw suite 800 washington , dc 20036 office 202-419-3412 mobile 202-577-5789 www.shapiroglobal.com b ' john , sorry we didn ’ t get to visit this morning . i teach once a semester for madeleine , and this be the morning , so i have to exit early . it be great that you be there for kati

In [308]:
%time clean(podesta.head().msg_body[1])

CPU times: user 7min 52s, sys: 3.46 s, total: 7min 56s
Wall time: 8min 3s


"bjohn , sorry we do n't get to visit this morning . i teach once a semester for madeleine , and this be the morning , so i have to exit early . it be great that you be there for katie , but i 'm not surprise . having be the beneficiary of your support when i run for congress , i know that you 're always there for former colleague and old friend . the republicans seem to be move along their remarkable path , set up a clash between one of the least qualified presidential candidate in history with one of the most qualified . given the state of the country , it promise to be quite a battle . one that we have to win . all my best , ira ira shapiro ira shapiro global strategies , llc 1200 new hampshire avenue , nw suite 800 washington , dc 20036 office 202-419-3412 mobile 202-577-5789 www.shapiroglobal.com b ' john , sorry we didn ’ t get to visit this morning . i teach once a semester for madeleine , and this be the morning , so i have to exit early . it be great that you be there for kati

In [82]:
# runs 50% faster when scheduler is provided for compute
# this is not used to parallelize the code and not loading the entire dataframe into memory, which apply does
# %time clean = podesta.apply(lambda df: clean(df.msg_body), meta=pd.Series(dtype='object', name='clean_body'), axis=1)

# clean_body = clean.compute(scheduler='processes')


In [307]:
%time podesta['clean_body'] = podesta.map_partitions(lambda df: df.msg_body.apply(clean), meta=pd.Series(dtype='object', name='clean_body'))

podesta.head(2)

CPU times: user 5.69 s, sys: 222 ms, total: 5.91 s
Wall time: 5.96 s


,date,from,to,subject,originator_org,spam_header,msg_body,clean_body
0,"Fri, 3 Apr 2015 14:06:30 +0000",Tina Flournoy <Tina@presidentclinton.com>,John Podesta <john.podesta@gmail.com>,just talked to him,NaN,NaN,b'\n\nSC was the first thing he mentioned.\nTo...,b ' sc be the first thing he mention . told hi...
1,"Mon, 22 Feb 2016 23:00:30 +0000",Ira Shapiro <ira@shapiroglobal.com>,John Podesta <john.podesta@gmail.com>,today's fundraiser,shapiroglobal.com,NSPM,"Noneb""John,\n\nSorry we didn't get to visit th...","bjohn , sorry we do n't get to visit this morn..."


In [61]:
# podesta.loc[~podesta.spam_header.isna()].head()

In [84]:
%who

BeautifulSoup	 clean	 clean_map	 copy	 csv	 cwd	 dd	 dtypes	 extract	 
get	 html	 mailbox	 os	 pd	 podesta	 re	 time	 unicodedata	 



In [86]:
import gc
gc.collect()

183